<a href="https://colab.research.google.com/github/YoheiFukuhara/keras-for-beginner/blob/master/Keras07_hyperas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ブログ記事[「Hyperasを使ったKerasハイパーパラメータチューニング」](https://qiita.com/FukuharaYohei/items/c8cc9d80452350b449eb)にて解説。

# ライブラリインポート

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from hyperopt import Trials, STATUS_OK, tpe, rand
from hyperas import optim
from hyperas.distributions import choice, uniform

#訓練データ設定

In [0]:
def data():
    import numpy as np
    x_train = np.random.rand(128, 2)
    y_train = (np.sum(x_train, axis=1) > 1.0) * 1
    y_train = y_train.reshape(128,1)
    
    return x_train, y_train

# モデル定義

In [0]:
def create_model(x_train, y_train):
    # Sequentialモデル使用(Sequentialモデルはレイヤを順に重ねたモデル)
    model = Sequential()

    # 結合層
    model.add(Dense({{choice([8, 16, 32, 64, 128])}}, input_dim=2, activation="tanh"))
    
    model.add(Dropout({{uniform(0, 1)}}))
#    model.add(Dropout(0.2))

    # 結合層：入力次元を省略すると自動的に前の層の出力次元数を引き継ぐ
    model.add(Dense(1, activation="sigmoid"))

    # モデルをコンパイル
    model.compile(loss="binary_crossentropy",
                  optimizer={{choice(['adam', 'sgd'])}},
                  metrics=["accuracy"])
    
    result = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=0)
    
    # 普通はここでevaluateしてその結果をチューニングに使うが、今回は簡易的に訓練時の結果を使用
    
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

# ハイパーパラメータチューニング実行

In [0]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=3,
                                      eval_space=True,
                                      notebook_name='Keras07_hyperas', # This is important!
                                      trials=Trials())

>>> Imports:
#coding=utf-8

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Dropout
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe, rand
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import numpy as np
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [8, 16, 32, 64, 128]),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'optimizer': hp.choice('optimizer', ['adam', 'sgd']),
    }

>>> Data
  1: 
  2: import numpy as np
  3: x_train = np.random.rand(128, 2)
  4: y_train = (np.sum(x_train, axis=1) > 1.0) * 1
  5: y_train = y_train.reshape(128,1)
  6: 
  7: 
  8: 
  9: 
>>> Resulting replaced keras model:

  1: def keras_fmin_fnct(space):
  2: 
  3:     # Sequentialモデル使用(Sequentialモデルはレイヤを順に重ねたモデル)
  4:     mode

In [0]:
x_test, y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(x_test, y_test))

Evalutation of best performing model:
128/128 [==============================] - 0s 99us/sample - loss: 0.4846 - acc: 0.9141
[0.4846123978495598, 0.9140625]


In [0]:
print("Best performing model chosen hyper-parameters:")
#print(best_run)
best_run

Best performing model chosen hyper-parameters:


{'Dense': 128, 'Dropout': 0.23316134447477344, 'optimizer': 'adam'}